In [6]:
## Receive the signal
include("rxsignal_withchannelandfreqoff.jl");

In [7]:
## Load the template signal
include("pss2.jl");

In [8]:
using FFTW

In [9]:
## Load the useful math operations
include("operations.jl");

In [10]:
## Assign the received signal a variable Ŝᵣₓ³
𝑅ₚₛₛ³ = rxs3; # RX Received Signal 3 File Handle
@show size(𝑅ₚₛₛ³); 

size(𝑅ₚₛₛ³) = (616447, 1)


In [11]:
using LinearAlgebra

In [12]:
## Assign the template signal a variable 𝐻ₚₛₛ²
## convert the signal in time domain
Pₚₛₛ² = pss_2; # File Handle
Hₚₛₛ²ᵀ = transpose.(Pₚₛₛ²); # Hessian Transpose ?
𝐻ₚₛₛ² = Fᴵ(Hₚₛₛ²ᵀ); # S Slanted (fourier transform) in time domain
𝐻ₚₛₛ² ./= norm(𝐻ₚₛₛ²);
𝐻ₚₛₛ² = 𝐻ₚₛₛ²[(end-143):end] ⧺ 𝐻ₚₛₛ² ; @show size(𝐻ₚₛₛ²); # end is 2048 in this case, concat math 

size(𝐻ₚₛₛ²) = (2192,)


In [13]:
## Prepare the template signal for convolution
𝐻̅ₚₛₛ² = ⦰(𝐻ₚₛₛ²); # reverse #typed H\overbar
𝐻̅ₚₛₛ²ᴴ = conj(𝐻̅ₚₛₛ²); # conjugate 

In [14]:
using DSP

In [15]:
## Perform the convolution between the 2 signals
𝐻̂ₚₛₛ² = 10 * log10.(abs.( 𝑅ₚₛₛ³ ⊗ 𝐻̅ₚₛₛ²ᴴ )); @show size( 𝐻̂ₚₛₛ² );  #typed \itH\hat

size(𝐻̂ₚₛₛ²) = (618638, 1)


In [16]:
# Find maximum value and its index
Ĉᵩ², 𝑁̂𝑓² = argmax(𝐻̂ₚₛₛ²); @show Ĉᵩ², 𝑁̂𝑓²;

(Ĉᵩ², 𝑁̂𝑓²) = (50.649042476081405, CartesianIndex(6628, 1))


In [17]:
θ₁ = 144
θ₂ = 2048
getindex(𝑁̂𝑓², 1) - ( θ₂ + θ₁ )

4436

In [18]:
using Plots

In [19]:
# Plot the result
m2_chan_plot = plot(𝐻̂ₚₛₛ², xlabel="Sample", ylabel="Power (dB)", title="Convolution Result", ylim=(-20, 60))
savefig(m2_chan_plot,"images/m2_chan_plot.png");

<img src=images/m2_chan_plot.png width='' heigth='' > </img>

In [20]:
########################################################
# Frequency Offset Estimator Function
########################################################
function freq_offset_est(𝑅ₚₛₛ, 𝐻ₚₛₛ, Nf, m, 𝑓ₛ)

    # Frequency offset estimator
    Y = zeros(ComplexF64, length(m));
    L = length(𝐻ₚₛₛ)
    t = 0:(1/𝑓ₛ):((L-1)/𝑓ₛ)

    signal_part = 𝑅ₚₛₛ[Nf:(Nf + L -1)]; @show size( signal_part )
    for j = 1:length(m)
        Y[j] = Y[j] + abs.(∑(exp.(-2*pi*im*m[j].*t) .* conj(𝐻ₚₛₛ) .* ⦰(signal_part)))^2;
    end

    return Y
end

freq_offset_est (generic function with 1 method)

In [21]:
𝑓ₛ = 61.44e6 # Msamples/s.

Δ𝑓 = 10.0
m = -7500.:Δ𝑓:7500.

N𝑓 = getindex(𝑁̂𝑓², 1) - length(𝐻ₚₛₛ²) + 1; @show N𝑓;

N𝑓 = 4437


In [22]:
# Grab the function Profs Frequency Offset with his values
Y = freq_offset_est(𝑅ₚₛₛ³, 𝐻ₚₛₛ², N𝑓, m, 𝑓ₛ); # @show Y

size(signal_part) = (2192,)


In [23]:
N̂𝑓, m̂ = argmax(abs.(Y)); 
@show (10 * log10(N̂𝑓));
@show (m̂);

10 * log10(N̂𝑓) = 88.46888484427753
m̂ = 587


In [24]:
cfo_estim_plot = 
plot(10 * log10.(abs.(Y)), xlabel="Sample", ylabel="Power (dB)", title="Estimation Result")
scatter!((m̂, (10 * log10(N̂𝑓))), color="red", label="m̂")
savefig(cfo_estim_plot,"images/cfo_estim_plot.png");

<img src=images/cfo_estim_plot.png width='' height='' > </img>

In [42]:
# Compensate CFO by multiplying the received signal with a complex exponential
function compensate_cfo(𝑅ₚₛₛ, N̂𝑓, 𝑓ₛ, Npss)
    # Generate a complex exponential with the specified frequency offset
    𝑅̂ₚₛₛ = 𝑅ₚₛₛ .* exp.(-2π * im * collect(1:4*Npss) * (N̂𝑓/𝑓ₛ))
    return 𝑅̂ₚₛₛ
end

compensate_cfo (generic function with 2 methods)

In [64]:
(m̂+512)-m̂
Npss = θ₂ + θ₁

2192

In [65]:
𝑅̅ₚₛₛ = 𝑅ₚₛₛ³[N𝑓:(N𝑓+Npss)-1]; @show m̂, size(𝑅̅ₚₛₛ)

(m̂, size(𝑅̅ₚₛₛ)) = (587, (2192,))


(587, (2192,))

In [66]:
Y = compensate_cfo(𝑅̅ₚₛₛ, N̂𝑓, 𝑓ₛ, Npss)

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2192 and 8768

```julia
using Plots

# Example usage
y = d # Your complex signal here

# Extract real and imaginary components
I_values = real(y)
Q_values = imag(y)

# Plot I/Q constellation
scatter(I_values, Q_values, xlabel="In-phase (I)", ylabel="Quadrature (Q)", label="Constellation Points", ratio=1, legend=:bottomright)
```